# Preprocess Clinical Data

This notebook is used to read clinical data (in tabular format) from S3 and store features in SageMaker FeatureStore.

## Step 1: Read in the SageMaker JumpStart Solution configuration

In [26]:
import json

SOLUTION_CONFIG = json.load(open("stack_outputs.json"))
SOLUTION_BUCKET = SOLUTION_CONFIG["SolutionS3Bucket"]
REGION = SOLUTION_CONFIG["AWSRegion"]
SOLUTION_PREFIX = SOLUTION_CONFIG["SolutionPrefix"]
SOLUTION_NAME = SOLUTION_CONFIG["SolutionName"]
BUCKET = SOLUTION_CONFIG["S3Bucket"]
ROLE = SOLUTION_CONFIG["IamRole"]

## Step 2: Download and read in the clinical dataset

In [27]:
from sagemaker.s3 import S3Downloader

input_data_bucket = f"s3://{SOLUTION_BUCKET}-{REGION}/{SOLUTION_NAME}/data"
print("solution data: ")
S3Downloader.list(input_data_bucket)

solution data: 


[]

#### Download the input data from S3

In [28]:
import pandas as pd

file_name = "NSCLCR01Radiogenomic_DATA_LABELS_2018-05-22_1500-shifted.csv"

input_data = f"{input_data_bucket}/{file_name}"
!aws s3 cp $input_data .

data_clinical = pd.read_csv(file_name)

## Step 3: Data Preprocessing

In [29]:
# Keep samples starting with "R01-*" as these IDs have corresponding medical imaging data. Delete samples with Case IDs "AMC-*". 
data_clinical = data_clinical[~data_clinical["Case ID"].str.contains("AMC")]

# Delete columns with ID and dates
list_delete_cols = ['Quit Smoking Year', 'Date of Recurrence', 'Date of Last Known Alive', 'Date of Death', 'CT Date', 'PET Date']
data_clinical.drop(list_delete_cols, axis=1, inplace=True)

# List of features with catergorical value
list_encode_cols = ["Patient affiliation", "Gender", "Ethnicity", "Smoking status", "%GG", "Tumor Location (choice=RUL)", "Tumor Location (choice=RML)", "Tumor Location (choice=RLL)", "Tumor Location (choice=LUL)", "Tumor Location (choice=LLL)", "Tumor Location (choice=L Lingula)", "Tumor Location (choice=Unknown)", "Histology ", "Pathological T stage", "Pathological N stage", "Pathological M stage", "Histopathological Grade", "Lymphovascular invasion", "Pleural invasion (elastic, visceral, or parietal)", "EGFR mutation status", "KRAS mutation status", "ALK translocation status", "Adjuvant Treatment", "Chemotherapy", "Radiation", "Recurrence", "Recurrence Location"]

# List of features with numeric value
list_nonenc_cols = ["Case ID", "Age at Histological Diagnosis", "Weight (lbs)", "Pack Years", "Time to Death (days)", "Days between CT and surgery", "Survival Status"]

# One-hot encoding of features with categorical value
data_clinical_enc = pd.get_dummies(data_clinical[list_encode_cols])

data_clinical_nonenc = data_clinical[list_nonenc_cols]

# Combine all features
data_clin = pd.concat([data_clinical_enc, data_clinical_nonenc], axis=1)

# Feature names inside FeatureStore should not have special chars and should be < 64 chars long
# Update feature names accordingly

l_char = ['-',' ','%','/','<','>','(',')','=',',',':']

for col in (data_clin.columns):

    if (col == "Case ID"):
        data_clin.rename(columns={col: col.replace(' ','_')}, inplace = True)
        continue

    for char in l_char:
        if char in col:
            data_clin.rename(columns={col: col.replace(char,'')}, inplace = True)
            col = col.replace(char,'')
            
    if (len(col)>=64):
        data_clin.rename(columns={col: col[:60]}, inplace = True)
        
# Change label (survival status) "Dead"=1 and "Alive"=0 
data_clin["SurvivalStatus"].replace({"Dead": "1", "Alive": "0"}, inplace=True)


# Drop samples with missing values. 
# Fill NaN with 0. For eg. PackYears for non-smokers is "NA". Change it to 0.
data_clin = data_clin[data_clin['Weightlbs'] != "Not Collected"]
data_clin = data_clin[data_clin['PackYears'] != "Not Collected"]
data_clin.fillna(0)

,Patientaffiliation_Stanford,Patientaffiliation_VA,Gender_Female,Gender_Male,Ethnicity_AfricanAmerican,Ethnicity_Asian,Ethnicity_Caucasian,Ethnicity_HispanicLatino,Ethnicity_NativeHawaiianPacificIslander,Smokingstatus_Current,...,RecurrenceLocation_distant,RecurrenceLocation_local,RecurrenceLocation_regional,Case_ID,AgeatHistologicalDiagnosis,Weightlbs,PackYears,TimetoDeathdays,DaysbetweenCTandsurgery,SurvivalStatus
49,1,0,1,0,0,0,1,0,0,0,...,0,0,0,R01-001,79,146,37,0.0,42,0
50,1,0,1,0,0,1,0,0,0,0,...,0,0,0,R01-002,65,195,0,0.0,23,0
53,1,0,0,1,0,0,0,0,1,0,...,1,0,0,R01-005,84,145,20,1456.0,92,1
56,1,0,0,1,0,0,1,0,0,0,...,0,0,0,R01-008,73,102,0,0.0,34,0
57,0,1,0,1,0,0,1,0,0,1,...,0,0,0,R01-009,59,133,100,1344.0,59,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,1,0,0,1,0,0,0,1,0,1,...,0,0,0,R01-158,67,143,25,0.0,5,0
206,1,0,0,1,0,0,1,0,0,0,...,0,0,0,R01-159,75,184,55,0.0,14,0
207,0,1,0,1,0,0,1,0,0,0,...,0,0,0,R01-160,61,231.5,12,0.0,72,0
209,1,0,0,1,0,1,0,0,0,0,...,0,0,0,R01-162,67,158,15,671.0,65,1


## Step 4: Create SageMaker FeatureStore

Firstly, we cast the object dtype to string which will then map to String feature type in the SageMaker FeatureStore. We add `record_identifier_feature_name` and `event_time_feature_name` columns to the dataset for creating the feature store.

In [30]:
import time

def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        print (label)
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")
            
current_time_sec = int(round(time.time()))

# Cast object dtype to string. SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(data_clin)

# Record identifier and event time feature names
record_identifier_feature_name = "Case_ID"
event_time_feature_name = "EventTime"

# Append EventTime feature
data_clin[event_time_feature_name] = pd.Series([current_time_sec]*len(data_clin), dtype="float64")

## If event time generates NaN
data_clin[event_time_feature_name] = data_clin[event_time_feature_name].fillna(0)

Patientaffiliation_Stanford
Patientaffiliation_VA
Gender_Female
Gender_Male
Ethnicity_AfricanAmerican
Ethnicity_Asian
Ethnicity_Caucasian
Ethnicity_HispanicLatino
Ethnicity_NativeHawaiianPacificIslander
Smokingstatus_Current
Smokingstatus_Former
Smokingstatus_Nonsmoker
GG_0
GG_100
GG_2550
GG_5075
GG_75100
GG_025
GG_NotAssessed
TumorLocationchoiceRUL_Checked
TumorLocationchoiceRUL_Unchecked
TumorLocationchoiceRML_Checked
TumorLocationchoiceRML_Unchecked
TumorLocationchoiceRLL_Checked
TumorLocationchoiceRLL_Unchecked
TumorLocationchoiceLUL_Checked
TumorLocationchoiceLUL_Unchecked
TumorLocationchoiceLLL_Checked
TumorLocationchoiceLLL_Unchecked
TumorLocationchoiceLLingula_Checked
TumorLocationchoiceLLingula_Unchecked
TumorLocationchoiceUnknown_Unchecked
Histology_Adenocarcinoma
Histology_NSCLCNOSnototherwisespecified
Histology_Squamouscellcarcinoma
PathologicalTstage_T1a
PathologicalTstage_T1b
PathologicalTstage_T2a
PathologicalTstage_T2b
PathologicalTstage_T3
PathologicalTstage_T4
Patholo

Next step, we define the FeatureGroup and load feature definitions to the feature group.

In [31]:
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup


boto_session = boto3.Session(region_name=REGION)
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=REGION)
featurestore_runtime = boto_session.client(service_name="sagemaker-featurestore-runtime", region_name=REGION)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

clinical_feature_group_name = f"{SOLUTION_PREFIX}-clinical-feature-group"
%store clinical_feature_group_name
clinical_feature_group = FeatureGroup(name=clinical_feature_group_name, sagemaker_session=feature_store_session)

# Load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
clinical_feature_group.load_feature_definitions(data_frame=data_clin) # output is suppressed

We create the FeatureGroup for the clinical dataset with both online and offline store enabled.

In [32]:
%%time

def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")
    
prefix = "sagemaker-featurestore-demo"

clinical_feature_group.create(
    s3_uri=f"s3://{BUCKET}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=ROLE,
    enable_online_store=True
)

wait_for_feature_group_creation_complete(feature_group=clinical_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup sagemaker-soln-lcsp-js-abc-clinical-feature-group successfully created.
CPU times: user 29 ms, sys: 0 ns, total: 29 ms
Wall time: 15.6 s


After the feature group is created, we can ingest the clinical dataset to its feature group.

In [33]:
clinical_feature_group.ingest(
    data_frame=data_clin, max_workers=3, wait=True
)

IngestionManagerPandas(feature_group_name='sagemaker-soln-lcsp-js-abc-clinical-feature-group', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7ff3f869ead0>, max_workers=3, max_processes=1, _async_result=<multiprocess.pool.MapResult object at 0x7ff3f91eedd0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

## Next Stage

Next, we'll take a look at preparing the clinical data.

Click here to [continue](./3_preprocess_imaging_data.ipynb).